In [ ]:
import numpy as np
import tensorflow as tf
import random as python_random

from tensorflow import keras
from tensorflow.keras import datasets
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import classification_report
import datetime
import os
# for reproducibility purposes
np.random.seed(42)
python_random.seed(42)
tf.random.set_seed(42)

# load tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs

In [ ]:
def make_directories():
    d = datetime.datetime.today()
    timestamp = d.strftime('%Y%m%d_%H%M%S')
    # folder to store the tensorboard logs
    tensorlog_folder = os.path.join(os.path.curdir, 'logs', timestamp)
    # folder to store the trained models
    checkpoint_folder = os.path.join(os.path.curdir, 'models', timestamp)

    os.mkdir(tensorlog_folder)
    os.mkdir(checkpoint_folder)

    return checkpoint_folder, tensorlog_folder

In [ ]:
# 0: T-shirt/top, 1: Trouser, 2: Pullover, 3: Dress, 4: Coat, 5: Sandal, 6: Shirt, 7: Sneaker, 8: Bag, 9: Ankle boot
data = datasets.fashion_mnist
(train_imgs, train_lbls), (test_imgs, test_lbls) = data.load_data()

print(train_imgs.shape)
print(test_imgs.shape)

In [ ]:
# normalize the image data
train_imgs = train_imgs.astype('float32') / 255
test_imgs = test_imgs.astype('float32') / 255

print(Counter(train_lbls))
print(Counter(test_lbls))

In [ ]:
# print an image
print(train_lbls[9])
image = train_imgs[9]
plt.imshow(image, cmap=plt.cm.binary)
plt.show()

In [ ]:
# using the keras sequential model
model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu', name='dense_1'),
    layers.Dense(10, activation='softmax', name='dense_out')
])

# print the model summary
print(model.summary())

# compile the model
optimizer = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# get the required directories
check_dir, tboard_dir = make_directories()

# using the callbacks
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoints = keras.callbacks.ModelCheckpoint(
    filepath=check_dir,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False
)

tensorboard = keras.callbacks.TensorBoard(
    log_dir=tboard_dir
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    min_lr=1e-5,
    verbose=1
)

# fit the model with training data with a validation split
# by default .fit will shuffle the data
model.fit(
    train_imgs, 
    train_lbls, 
    validation_split=0.3, 
    epochs=10, 
    batch_size=64, 
    callbacks=[reduce_lr, early_stop, checkpoints, tensorboard]
)


In [ ]:
# evaluate the model on the test data
test_evals = model.evaluate(test_imgs, test_lbls)
print(test_evals)
# predict on the test data
predictions = model.predict(test_imgs)
predicted_lbls = np.argmax(predictions, axis=1)
cr = classification_report(test_lbls, predicted_lbls)
print(cr)

In [ ]:
# limitations with the sequential model
# single input and output
# sequence of layers

# using the functional API
# can be used with multiple inputs and outputs

# using the keras functional API
inputs = keras.Input(shape=(28,28))
x = layers.Flatten()(inputs)
x = layers.Dense(128, activation='relu', name='dense_1')(x)

outputs = layers.Dense(10, activation='softmax', name='dense_out')(x)

# creating the Model by grouping the layers
model = keras.Model(inputs=inputs, outputs=outputs)

# print the model summary
print(model.summary())

# compile the model
optimizer = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# get the required directories
check_dir, tboard_dir = make_directories()

# using the callbacks
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoints = keras.callbacks.ModelCheckpoint(
    filepath=check_dir,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False
)

tensorboard = keras.callbacks.TensorBoard(
    log_dir=tboard_dir
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    min_lr=1e-5,
    verbose=1
)

# fit the model with training data with a validation split
# by default .fit will shuffle the data
model.fit(
    train_imgs, 
    train_lbls, 
    validation_split=0.3, 
    epochs=10, 
    batch_size=64, 
    callbacks=[reduce_lr, early_stop, checkpoints, tensorboard]
)


In [ ]:
# evaluate the model on the test data
test_evals = model.evaluate(test_imgs, test_lbls)
print(test_evals)
# predict on the test data
predictions = model.predict(test_imgs)
predicted_lbls = np.argmax(predictions, axis=1)
cr = classification_report(test_lbls, predicted_lbls)
print(cr)

In [ ]:
data = datasets.fashion_mnist
(train_imgs, train_lbls), (test_imgs, test_lbls) = data.load_data()

# reshape and normalize data for CNN
train_imgs = train_imgs.reshape((60000, 28, 28, 1)) 
train_imgs = train_imgs.astype("float32") / 255 
test_imgs = test_imgs.reshape((10000, 28, 28, 1)) 
test_imgs = test_imgs.astype("float32") / 255 

In [ ]:
inputs = keras.Input(shape=(28, 28, 1)) 
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs) 
x = layers.MaxPooling2D(pool_size=2)(x) 
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x) 
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.Flatten()(x)

outputs = layers.Dense(10, activation="softmax")(x) 

model = keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

# compile the model
optimizer = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# get the required directories
check_dir, tboard_dir = make_directories()

# using the callbacks
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoints = keras.callbacks.ModelCheckpoint(
    filepath=check_dir,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False
)

tensorboard = keras.callbacks.TensorBoard(
    log_dir=tboard_dir
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    min_lr=1e-5,
    verbose=1
)

# fit the model with training data with a validation split
# by default .fit will shuffle the data
model.fit(
    train_imgs, 
    train_lbls, 
    validation_split=0.3, 
    epochs=10, 
    batch_size=64, 
    callbacks=[reduce_lr, early_stop, checkpoints, tensorboard]
)

In [ ]:
# evaluate the model on the test data
test_evals = model.evaluate(test_imgs, test_lbls)
print(test_evals)
# predict on the test data
predictions = model.predict(test_imgs)
predicted_lbls = np.argmax(predictions, axis=1)
cr = classification_report(test_lbls, predicted_lbls)
print(cr)

* Use "image_dataset_from_directory" to read, shuffle, resize, batch, etc.. to load images
* In case the model overfits, use different data augmentation strategies such as: RandomFlip, RandomRotation, RandomZoom
* In case your dataset is small, use a pretrained model (i.e., if the model is well generalized)
    * e.g., ImageNet, ResNet